In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, FunctionTransformer, Imputer
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, FunctionTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler, Imputer, FunctionTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report



sns.set_style('whitegrid')
%config InlineBackend.figure_format = 'retina'
# %matplotlib.style.use('ggplot') 
%matplotlib inline
#created a one stop shop for all libraries that I have used in the past hence the abundance of them

In [46]:
# IMPORTS
results = pd.read_csv('results.csv')
races = pd.read_csv('races.csv',sep=',')
drivers = pd.read_csv('drivers.csv',encoding='latin-1')
seasons = pd.read_csv('seasons.csv',sep=',')
circuits = pd.read_csv('circuits.csv',encoding='latin-1')
constructors = pd.read_csv('constructors.csv',sep = ',')
constructor_results = pd.read_csv('constructorResults.csv',sep = ',')
laptimes = pd.read_csv('lapTimes.csv',sep = ',')
qual = pd.read_csv('qualifying.csv',sep = ',')

In [47]:
results.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22.0,1,1.0,1,1,10.0,58,34:50.6,5690616.0,39.0,2.0,01:27.5,218.3,1
1,2,18,2,2,3.0,5,2.0,2,2,8.0,58,5.478,5696094.0,41.0,3.0,01:27.7,217.586,1
2,3,18,3,3,7.0,7,3.0,3,3,6.0,58,8.163,5698779.0,41.0,5.0,01:28.1,216.719,1
3,4,18,4,4,5.0,11,4.0,4,4,5.0,58,17.181,5707797.0,58.0,7.0,01:28.6,215.464,1
4,5,18,5,1,23.0,3,5.0,5,5,4.0,58,18.014,5708630.0,43.0,1.0,01:27.4,218.385,1


In [48]:
# MERGES

results_with_loc_and_year = pd.merge(results, races, on = 'raceId' )
results_with_loc_and_year.rename(columns={'driverId_x': 'driverId'}, inplace=True)
results_with_loc_and_year.shape
results_by_driver = pd.merge(results_with_loc_and_year, drivers, on = 'driverId' )
results_by_driver.shape
results_by_driver.rename(columns={'constructorId_x': 'constructorIdd'}, inplace=True)
#merged results_by_driver df and constructors df to include constructors names within main dataframe
results_by_driver_team = pd.merge(results_by_driver, constructors, on = 'constructorId')
#renamed columns after merge process
results_by_driver_team.rename(columns={'name_x': 'grand_prix_name', 'name_y': 'Team_Name'}, inplace=True)


In [49]:
df = results_by_driver_team.drop(labels = ['Probability','url_y','url'], axis = 1)
#taking data from 1985 onwards
df = results_by_driver_team[(results_by_driver_team['year'] >= 1985)]
df.shape

(13702, 43)

In [50]:
# rename various columns
df.rename(columns={'constructorId': 'Team_Code'}, inplace=True)




/Users/jameslambert/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [51]:
def sheitzer(pr):

  if pr['Team_Code'] == 1:   #setting up Mclaren based on Team_Code 
    return '1.0'
  if pr['Team_Code'] == 9:  #setting up RedBull based on Team_Code
    return '1.0'
  if pr['Team_Code'] == 131: #setting up Mercedes based on Team_Code
    return '1.0'
  if pr['Team_Code'] == 6:  #setting up Ferrari based on Team_Code
    return '1.0'
  else:
    return '0.50'
df["Probability"] = df.apply(sheitzer, axis = 1)

/Users/jameslambert/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


DEALING WITH NULLS

In [52]:
# see the non numeric column names
df.columns.to_series().groupby(df.dtypes).groups

{dtype('int64'): Index(['resultId', 'raceId', 'driverId', 'Team_Code', 'grid', 'positionOrder',
        'laps', 'statusId', 'year', 'round', 'circuitId'],
       dtype='object'),
 dtype('float64'): Index(['number_x', 'position', 'points', 'milliseconds', 'fastestLap', 'rank',
        'number_y', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
        'Unnamed: 10'],
       dtype='object'),
 dtype('O'): Index(['positionText', 'time_x', 'fastestLapTime', 'fastestLapSpeed',
        'grand_prix_name', 'date', 'time_y', 'url_x', 'driverRef', 'code',
        'forename', 'surname', 'dob', 'nationality_x', 'url_y',
        'constructorRef', 'Team_Name', 'nationality_y', 'url', 'Probability'],
       dtype='object')}

In [53]:
qual_null_fills=['positionText', 'time_x', 'fastestLapTime', 'fastestLapSpeed',
        'grand_prix_name', 'date', 'time_y', 'url_x', 'driverRef', 'code',
        'forename', 'surname', 'dob', 'nationality_x', 'url_y',
        'constructorRef', 'Team_Name', 'nationality_y', 'url', 'Probability']
for i in qual_null_fills:
    filtering=df[i].isnull()
    df.loc[filtering,i]='Nones'   # i is for the coolumn wiithout it fucks it up
 #dominant type

/Users/jameslambert/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [54]:
# df.select_dtypes(exclude = [np.number]).columns

In [55]:
# drop rows that are non-important to my model
df = df.drop(labels = ['Unnamed: 6', 'Unnamed: 7','Unnamed: 8', 'Unnamed: 9', \
                       'Unnamed: 10','url','url_y'], axis =1)
#replace nulls with 0
df.fillna(0, inplace=True)

# EDA

In [56]:
df.shape

(13702, 36)

In [57]:
df.columns
# rename columns for modeling
df.rename(columns={'Team_Code':'constructorRef'}, inplace=True)

In [ ]:
df_dummies = pd.get_dummies(df, columns = ['positionText'])

In [ ]:
df_dummies_corr = df_dummies.corr().sort_values('rank', ascending=False)
df_dummies_corr

In [ ]:
plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(15,15)) 
sns.heatmap(df_dummies_corr, xticklabels=df_dummies_corr.columns, yticklabels=df_dummies_corr.columns, ax=ax)

In [ ]:
# df_dummies.columns

In [ ]:
#non numeric column names
df_dummies.columns.to_series().groupby(df_dummies.dtypes).groups

In [ ]:
df1 = df_dummies.drop(labels = ['positionText_11','positionText_12', 'positionText_13', 'positionText_14',
        'positionText_15', 'positionText_16', 'positionText_17',
        'positionText_18', 'positionText_19','positionText_20', 'positionText_21', 
        'positionText_22','positionText_23', 'positionText_24','positionText_D', 
        'positionText_E','positionText_F', 'positionText_N', 'positionText_R', 
        'positionText_W','Team_Name','constructorRef','nationality_y','nationality_x','dob',\
        'surname','forename','time_x', 'fastestLapTime', 'fastestLapSpeed', 'grand_prix_name',
        'date', 'time_y', 'url_x', 'driverRef', 'code', 'Probability'], axis = 1)

In [ ]:
df_dummies = df1

In [ ]:
df_dummies_corr_ovfa = df_dummies.corr().sort_values('rank', ascending=False)
df_dummies_corr_ovfa


In [ ]:
plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(15,15)) 
sns.heatmap(df_dummies_corr_ovfa, xticklabels=df_dummies_corr_ovfa.columns, yticklabels=df_dummies_corr_ovfa.columns, ax=ax)

In [ ]:
df_dummies.to_csv('team_stats.csv', index=False)

# LR MODEL

In [ ]:
#set-up training df for modeling
df_dummies_train = df_dummies[(df_dummies['year'] <= 2016)]

In [ ]:
numerical_features = list(df_dummies_train._get_numeric_data().drop(['rank'], axis = 1).columns)

In [ ]:
features = numerical_features

In [ ]:
df_dummies_train.shape

In [ ]:
# df_dummies_test = (df_dummies['year']== 2017)
#set-up testing df for modeling 
df_dummies_test = df_dummies[(df_dummies['year'] == 2017)]
df1 = df_dummies[(df_dummies['year'] == 2017)]

In [ ]:
test_features = list(df_dummies_test._get_numeric_data().drop(['rank'], axis = 1).columns)

In [ ]:
df_dummies_test.shape
# df_dummies_train.shape

In [ ]:
X = df_dummies_train[features]
y = df_dummies_train['rank']

            
X_train, X_test, y_train, y_test = train_test_split(X[features], y, \
                                                    random_state=42, test_size = .5)

# use euclidian distance 
# msd
# mse
params = {}
gs = GridSearchCV(RandomForestRegressor(100), 
                  param_grid = params)
gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

In [63]:
from sklearn.metrics import r2_score
  

In [ ]:
gs.score(X_test, y_test)
pred = gs.predict(df_dummies_test[features])
df_dummies_test['rank'] = pred

In [ ]:
df_dummies_test.to_csv('fucker.csv')

In [ ]:
df_dummies_test[['round']].to_csv('check_reg_model.csv', index = False)

# LOGISTIC REGRESSION


In [ ]:
train = df_dummies_train
test = df_dummies_test

In [ ]:
target_corr = abs(train.corr()[['rank']]).sort_values(by='rank',ascending=False)
target_corr.drop('rank',axis=0,inplace=True)

In [ ]:
#non numeric column names
df_dummies.columns.to_series().groupby(df_dummies.dtypes).groups

In [ ]:
train.columns.to_series().groupby(train.dtypes).groups

In [ ]:
target_corr.head(20).index

In [ ]:
lr = LogisticRegression()
y = train['rank']
X = train.drop('rank', axis = 1)
lr.fit(X, y)

In [ ]:
results = lr.predict_proba(test)

# list of prediction probability that class is 1
results_1_probas = [prediction[1] for prediction in results]

# submission = test[['rank']]
# submission['rank'] = results_1_probas

In [ ]:
submission.to_csv('../submissions/submission1.csv', index_label=False, index=False)

In [ ]:
test_sub = pd.read_csv('../submissions/submission1.csv')

test_sub.head()

# ADA BOOSTING MODEL (not as good as Random Forest)

In [ ]:
ada = AdaBoostClassifier(random_state = 42)
ada_params = {
    'n_estimators': [40]
}
gs_ada = GridSearchCV(ada, ada_params)
gs_ada.fit(X_train, y_train)
print(gs_ada.best_score_)
gs_ada.best_params_

In [ ]:
gs_ada.score(X_test, y_test)

# Gradient Boosting

In [ ]:
# gb = GradientBoostingClassifier()
# gb_params = {}
# gs_gb = GridSearchCV(gb, gb_params)
# gs_gb.fit(X_train, y_train)
# print(gs_gb.best_score_)
# gs_gb.best_params_

In [ ]:
# gs_gb.score(X_test, y_test)

# Neural Network

In [ ]:
X = df_dummies_train[features]
y = df_dummies_train['rank']


X_train, X_test, y_train, y_test = train_test_split(X, y)



In [ ]:

ss = StandardScaler()

ss.fit(X_train)
X_train_sc = ss.fit_transform(X_train)
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [ ]:
model = Sequential()
model.add(Dense(3, input_dim = 72, activation = 'relu'))
model.add(Dense(1))   #set up your network


In [ ]:
model.compile(loss ='mean_squared_error', optimizer = 'adam')

In [ ]:
# model.fit(X_train_sc, y_train, validation_data = (X_test_sc, y_test), epochs = 100)